In [4]:
import numpy as np
import cv2

In [5]:
def apply_mask(face: np.array, mask: np.array) -> np.array:
    """Add the mask to the provided face, and return the face with mask."""
    mask_h, mask_w, _ = mask.shape
    face_h, face_w, _ = face.shape
    
    # Resize the mask to fit on face
    factor = min(face_h / mask_h, face_w / mask_w)
    
    new_mask_w = int(factor * mask_w)
    new_mask_h = int(factor * mask_h)
    new_mask_shape = (new_mask_w, new_mask_h)
    
    resized_mask = cv2.resize(mask, new_mask_shape)
    
    face_with_mask = face.copy()
    non_white_pixels = (resized_mask < 250).all(axis=2)
    
    off_h = int((face_h - new_mask_h) / 2)
    off_w = int((face_w - new_mask_w) / 2)
    
    face_with_mask[off_h: off_h+new_mask_h, off_w: off_w+new_mask_w][non_white_pixels] = \
         resized_mask[non_white_pixels]
    return face_with_mask

In [6]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

if cap.isOpened():
    ret,frame = cap.read()
else:
    ret = False

while ret:
    mask = cv2.imread('dog.png')
    ret,frame = cap.read()
    frame_h, frame_w, _ = frame.shape
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        y0, y1 = int(y - 0.25*h), int(y + 0.75*h)
        x0, x1 = x, x + w
        if x0 < 0 or y0 < 0 or x1 > frame_w or y1 > frame_h:
            continue
        frame[y0: y1, x0: x1] = apply_mask(frame[y0: y1, x0: x1], mask)
    
    cv2.imshow("CamFeed",frame)
    
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()